In [1]:
%pip install selenium

  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.7 MB 8.4 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.7 MB 12.6 MB/s eta 0:00:01
   ---------------------------

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import pandas as pd

def scrape_routes_buses(url,num_page):
    driver = webdriver.Chrome()
    driver.maximize_window()   
    driver.get(url)#give the url
    time.sleep(3)  # Wait for the page to load
    route_data = []

    # Try block for route extraction
    try:    
        for page in range(1, num_page + 1):
            if page > 1:
                # Wait for the pagination tab and click it
                pagination_tab = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']"))
                )
                driver.execute_script("arguments[0].click();", pagination_tab) #the click() did not work here
                time.sleep(3) 
            
            # Re-fetch the route elements after navigating to the new page
            routes = driver.find_elements(By.CLASS_NAME, 'route')

            for route in routes:
                try:
                    route_links = route.get_attribute('href')
                    route_names = route.text.strip()
                    route_data.append({"Route_Link": route_links, "Route_Name": route_names})
                except StaleElementReferenceException:
                    print("Element is not there, skipping...")

    except TimeoutException:
        print("Timed out during route extraction")

    # Extract bus details from route data
    bus_data = []

    for links in route_data:
        try:                         
            driver.get(links["Route_Link"])
            time.sleep(3)
            
            # Wait for the "View Buses" button and click it
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(3)  # Wait for buses to load
        except TimeoutException:
            print(f"Timeout: No button found for {links['Route_Link']}")

        try:
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait for the page to load more content
            
            # Extract bus data
            bus_names = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_types = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departure = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reach = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_ratings = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            
            # Handle seat availability using XPath
            seat_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            # Ensure all lists have the same length, otherwise handle missing data
            num_buses = len(bus_names)
            
            for i in range(num_buses):
                bus_detail = {
                    "Route_Name": links["Route_Name"],
                    "Route_Link": links["Route_Link"],
                    "Bus_Name": bus_names[i].text if i < len(bus_names) else '',
                    "Bus_Type": bus_types[i].text if i < len(bus_types) else '',
                    "Departure": departure[i].text if i < len(departure) else '',
                    "Duration": duration[i].text if i < len(duration) else '',
                    "Reach": reach[i].text if i < len(reach) else '',
                    "Star_Ratings": star_ratings[i].text if i < len(star_ratings) else '0',
                    "Price": price[i].text if i < len(price) else '',
                    "Seat_Availability": seat_available[i].text if i < len(seat_available) else '0'
                }
                bus_data.append(bus_detail)
        except Exception as e:
            print(f"Error extracting bus data for {links['Route_Link']}: {str(e)}")

       
        
    # Convert the bus_data to a pandas DataFrame
    df = pd.DataFrame(bus_data)
    # Quit the driver after scraping is complete
    driver.quit()
    # Check if df is empty
    if df.empty:
        print("No data was extracted.")
    else:
        print("Data extraction completed and saved to bus_details.csv") 

    return df

In [ ]:
#checking :

#df_kerala = scrape_routes_buses("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",2)
#print(df_kerala.head().to_csv("newdf.csv"))

Data extraction completed and saved to bus_details.csv
               Route_Name                                         Route_Link  \
0  Bangalore to Kozhikode  https://www.redbus.in/bus-tickets/bangalore-to...   
1  Bangalore to Kozhikode  https://www.redbus.in/bus-tickets/bangalore-to...   
2  Bangalore to Kozhikode  https://www.redbus.in/bus-tickets/bangalore-to...   
3  Bangalore to Kozhikode  https://www.redbus.in/bus-tickets/bangalore-to...   
4  Bangalore to Kozhikode  https://www.redbus.in/bus-tickets/bangalore-to...   

                Bus_Name                           Bus_Type Departure  \
0  KSRTC (Kerala) - 1378                      AC MULTI AXLE     13:46   
1   KSRTC (Kerala) - 129        SWIFT-GARUDA A/C SEATER BUS     14:15   
2   KSRTC (Kerala) - 235  Swift Deluxe Non AC Air Bus (2+2)     15:00   
3   KSRTC (Kerala) - 370                                        15:25   
4                                                                       

  Duration  Reach Star_Ra

In [3]:
df_kerala.head().to_csv("newdf.csv")